In [0]:
!pip install requirements.txt

In [0]:
dbutils.library.restartPython()

In [0]:
from databricks.vector_search.client import VectorSearchClient

In [0]:
#Create the endpoint
client = VectorSearchClient(service_principal_client_id=<CLIENT_ID>,service_principal_client_secret=<CLIENT_SECRET>)

client.create_endpoint(
    name="call_transcripts_vector_endpoint",
    endpoint_type="STANDARD"
)

In [0]:
transcripts = [
    (1, "Customer: Hi, I'm having trouble with my data plan. It seems to be running out faster than usual. Agent: I understand your concern. Let's take a look at your account and usage patterns to identify any issues."),
    (2, "Customer: Hello, I'd like to upgrade my phone. Can you tell me what options are available? Agent: Certainly! I'd be happy to go over our current phone upgrade options with you. What type of phone are you interested in?"),
    (3, "Customer: I'm getting poor signal at my new house. Is there anything that can be done? Agent: I'm sorry to hear that. Let's check the coverage in your area and see if there are any network improvements planned. Can you provide me with your new address?"),
    (4, "Customer: My bill seems higher than usual this month. Can you explain why? Agent: Of course, I'd be glad to review your bill with you. Let's go through the charges together and I'll explain any changes or additional fees."),
    (5, "Customer: I'm traveling abroad next week. How can I use my phone internationally? Agent: Great question! I can help you set up an international plan for your trip. First, let me know which countries you'll be visiting and for how long.")
]

transcripts_df = spark.createDataFrame(data, ["id", "call_transcript"])

(transcripts_df.write
  .format("delta")
  .mode("overwrite")
  .saveAsTable("dev.call_analytics.transcripts"))

In [0]:
index = client.create_delta_sync_index(
  endpoint_name="call_transcripts_vector_endpoint",
  source_table_name="dev.call_analytics.transcripts",
  index_name="dev.call_analytics_transcripts_index",
  pipeline_type="CONTINUOUS", #update the index anytime the source table changes
  primary_key="id",
  embedding_source_column="text",
  embedding_model_endpoint_name="e5-small-v2"
)